<div class="head0">
    <div class="head0__name">
        Examples for func module
    </div>
    <div class="head0__note">
        Fast multidimensional function interpolation by Chebyshev polynomials in the dense (numpy) or sparse (tensor train (TT) with cross approximation) format using Fast Fourier Transform (FFT).
    </div>
</div>

In [ ]:
import numpy as np

from fpcross import init_jupyter, Grid, Intertrain

init_jupyter()

<div class="end"></div>